# Sequential Uncertainty Reuction and Value of Information in Gaussian Process Models

We have a forest on a $25\times25$ grid, which means that it consists of $625$ units of cells, $(1,1), (1,2),\dots,(25,25)$. A farmer only has two choices $a = 0$,leave all cells for conservation, or $a = 1$, harvest all cells.

The uncertain profits on a cell $\mathbf{s} = (s_1, s_2)$ has $E[x(\mathbf{s})] = \mathbf{0}$, $\mathrm{Var}[x(\mathbf{s})] = 1^2$ and $\mathrm{Cov}[x(\mathbf{s}),x(\mathbf{s}')] = e^{-0.15h}$, where $h = \sqrt{||\mathbf{s}-\mathbf{s}'||^2}$. This yeilds the multivariate prior 
\begin{equation*}
\mathbf{x} \sim \mathcal{N}[\mathbf{0},\Sigma].
\end{equation*}

We do measurements along the North-South Lines in the grid. Which means that we measure $25$ cells at a time along the line $y(\mathbf{s}) = x(\mathbf{s}) + \mathcal{N}[0,5^2]$. This means we can measure $25$ different lines that are modelled by
\begin{equation*}
\mathbf{y}_j = \mathbf{F}_j\mathbf{x} + \mathcal{N}[\mathbf{0},\mathbf{T}].
\end{equation*}

All this yeilds the new mean and covariance at the $j$th line 
\begin{equation*}
\begin{array}{rcl}
\mathbf{\mu}_j & = & \mathbf{\mu} + \mathbf{\Sigma}\mathbf{F}_j^T(\mathbf{F}_j\mathbf{\Sigma}\mathbf{F}_j^T + \mathbf{T})^{-1}(\mathbf{y}_j - \mathbf{F}_j\mathbf{\mu})\\
\mathbf{\Sigma}_j & = & \mathbf{\Sigma} - \mathbf{\Sigma}\mathbf{F}_j^T(\mathbf{F}_j\mathbf{\Sigma}\mathbf{F}_j^T + \mathbf{T})^{-1}\mathbf{F_j}\mathbf{\Sigma}.
\end{array}
\end{equation*}

In [45]:
import numpy as np
N = 25
m = 625
def create_grid(m,N):
    Sigma = np.identity(m)
    s = np.zeros((2,m))
    T = np.identity(N)*5**2
    mu = np.zeros(m)
    for i in range(N):
        for j in range(N):
            s[:,j + i*N] = [i+1,j+1]
    for i in range(m-1):
        for j in range(i+1,m):
            Sigma[i,j] = Sigma[j,i] = np.exp(-0.15*np.sqrt((s[0,i] - s[0,j])**2 + (s[1,i] - s[1,j])**2))
    return s, mu, Sigma, T

def get_F_j(m,N,j):
    F_j = np.zeros((N,m))
    for i in range(N):
        for k in range(N):
            F_j[i,k*N + j-1] = 1
    return F_j

def get_Sigma_j(m,N,j,Sigma,T):
    F_j = get_F_j(m,N,j)
    tmp = (F_j@Sigma@F_j.transpose() + T)
    tmp = np.linalg.inv(tmp)
    Sigma_j = Sigma - Sigma@F_j.transpose()@tmp@F_j@Sigma
    return(Sigma_j)

In [48]:
s, mu, Sigma, T = create_grid(m,N)
Sigma_j = Sigma_j(m,N,2,Sigma,T)

### a)
Select the line j that m,aximises the reduction in the mean variance of the grid: i.e. the one that gives the smallest $\frac{1}{625}\sum_{k=1}^{625} \Sigma_j(k,k)$.


In [61]:
def max_reduc_mean_var(m,N,Sigma,T):
    tmp_min = np.inf
    tmp_min_j = np.nan
    for j in range(25):
        Sigma_j = get_Sigma_j(m,N,j,Sigma,T)
        tmp_mean_var = np.diag(Sigma_j).mean()
        if tmp_mean_var < tmp_min:
            tmp_min = tmp_mean_var
            tmp_min_j = j
    return tmp_min, tmp_min_j

In [68]:
min_mean_var, min_j = max_reduc_mean_var(m,N,Sigma,T)

print("Line {} maximizes the reduction in mean variance, and has magnitude {}.".format(min_j,min_mean_var))

Line 13 maximizes the reduction in mean variance, and has magnitude 0.830369561605575.
